In [ ]:
!pip install torch torchvision matplotlib scikit-learn


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import MNIST
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import time, os
from sklearn.cluster import KMeans


In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64*7*7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64*7*7)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [ ]:
transform = transforms.ToTensor()
train_data = MNIST(root='./data', train=True, download=True, transform=transform)
test_data = MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader_clean = DataLoader(test_data, batch_size=1000)

def add_noise(tensor, std=0.5):
    return torch.clip(tensor + torch.randn_like(tensor) * std, 0., 1.)

noisy_test_images = add_noise(test_data.data.unsqueeze(1).float() / 255.)
test_loader_noisy = DataLoader(TensorDataset(noisy_test_images, test_data.targets), batch_size=1000)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(model, loader, optimizer, criterion, epochs=3):
    model.train()
    for ep in range(epochs):
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    start = time.time()
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    end = time.time()
    return 100 * correct / total, end - start


In [ ]:
def apply_weight_sharing(model, bits=8):
    unique_weights = []
    for name, param in model.named_parameters():
        if 'weight' in name and len(param.data.size()) > 1:
            flat = param.data.cpu().numpy().flatten()
            unique_weights.extend(flat)

    unique_weights = np.array(unique_weights).reshape(-1, 1)
    kmeans = KMeans(n_clusters=2**bits, n_init=1).fit(unique_weights)
    centroids = kmeans.cluster_centers_

    with torch.no_grad():
        for name, param in model.named_parameters():
            if 'weight' in name and len(param.data.size()) > 1:
                shape = param.data.shape
                flat = param.data.cpu().numpy().flatten().reshape(-1, 1)
                clusters = kmeans.predict(flat)
                new_data = centroids[clusters].reshape(shape)
                param.data.copy_(torch.tensor(new_data, dtype=param.dtype))


In [ ]:
model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train(model, train_loader, optimizer, criterion, epochs=3)

# Apply weight sharing
apply_weight_sharing(model, bits=6)

# Evaluate
acc_clean, t_clean = evaluate(model, test_loader_clean)
acc_noisy, t_noisy = evaluate(model, test_loader_noisy)

torch.save(model.state_dict(), "ws_hash_model.pth")
size_mb = os.path.getsize("ws_hash_model.pth") / (1024 ** 2)

print(f"✅ Accuracy Clean: {acc_clean:.2f}% | Time: {t_clean:.2f}s")
print(f"✅ Accuracy Noisy: {acc_noisy:.2f}% | Time: {t_noisy:.2f}s")
print(f"📦 Model Size: {size_mb:.2f} MB")
